# **Generatívne súperiace siete a detekcia anomálií v oblasti bankových podvodov**

*Diplomová práca*

Bc. Laura Vištanová

Školiteľ: RNDr. Ľubomír Antoni, PhD. 

Dáta k diplomovej práci sme prevzali zo súťaže na stránke kaggle.com. Súťaž nájdeme na nasledujúcej stránke: https://www.kaggle.com/competitions/ieee-fraud-detection/overview

Ako základ sme si zobrali víťazov tejto súťaže (odkaz nižšie), predspracovanie dát sme prevzali od nich, všetky ostatné časti sú už ale výsledkom našej práce. 

#**Import knižníc**

In [ ]:
import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
 
import glob
from os import listdir
from os.path import isdir
from random import choice

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.feature_selection import SelectKBest
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

# ***Úprava dátovej sady***

Túto časť práce sme urobili podľa víťazného tímu súťaže. Pôvodné úpravy nájdeme na stránke: https://www.kaggle.com/code/cdeotte/xgb-fraud-with-magic-0-9600 

In [ ]:
BUILD95 = True
BUILD96 = True

import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# stĺpce so stringami 
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain','M1', 'M2', 'M3', 'M4','M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
str_type += ['id-12', 'id-15', 'id-16', 'id-23', 'id-27', 'id-28', 'id-29', 'id-30', 
            'id-31', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']

# prvých 53 stĺpcov
cols = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9']

# stĺpce v rozdelené podľa korelácie 
# https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id
v =  [1, 3, 4, 6, 8, 11]
v += [13, 14, 17, 20, 23, 26, 27, 30]
v += [36, 37, 40, 41, 44, 47, 48]
v += [54, 56, 59, 62, 65, 67, 68, 70]
v += [76, 78, 80, 82, 86, 88, 89, 91]

v += [107, 108, 111, 115, 117, 120, 121, 123] # možno skupina, neobsahuje NaN
v += [124, 127, 129, 130, 136] # koreluje so skupinami, neobsahuje NaN

# obsahujú veľa NaN 
v += [138, 139, 142, 147, 156, 162] #b1
v += [165, 160, 166] #b1
v += [178, 176, 173, 182] #b2
v += [187, 203, 205, 207, 215] #b2
v += [169, 171, 175, 180, 185, 188, 198, 210, 209] #b2
v += [218, 223, 224, 226, 228, 229, 235] #b3
v += [240, 258, 257, 253, 252, 260, 261] #b3
v += [264, 266, 267, 274, 277] #b3
v += [220, 221, 234, 238, 250, 271] #b3

v += [294, 284, 285, 286, 291, 297] # koreluje so skupinami, neobsahuje NaN
v += [303, 305, 307, 309, 310, 320] # koreluje so skupinami, neobsahuje NaN
v += [281, 283, 289, 296, 301, 314] # koreluje so skupinami, neobsahuje NaN

cols += ['V'+str(x) for x in v]
dtypes = {}
for c in cols+['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,34)]+\
    ['id-0'+str(x) for x in range(1,10)]+['id-'+str(x) for x in range(10,34)]:
        dtypes[c] = 'float32'
for c in str_type: dtypes[c] = 'category'

In [ ]:
#keby boli dáta na Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
%%time
# načítanie tréningových dát 
X_train = pd.read_csv('train_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols+['isFraud'])
train_id = pd.read_csv('train_identity.csv',index_col='TransactionID', dtype=dtypes)
X_train = X_train.merge(train_id, how='left', left_index=True, right_index=True)
# načítanie testovacích dát 
X_test = pd.read_csv('test_transaction.csv',index_col='TransactionID', dtype=dtypes, usecols=cols)
test_id = pd.read_csv('test_identity.csv',index_col='TransactionID', dtype=dtypes)
fix = {o:n for o, n in zip(test_id.columns, train_id.columns)}
test_id.rename(columns=fix, inplace=True)
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)
# target
y_train = X_train['isFraud'].copy()
del train_id, test_id, X_train['isFraud']; x = gc.collect()

print('Train shape',X_train.shape,'test shape',X_test.shape)

In [ ]:
# zobrazenie pôvodných D stĺpcov
plt.figure(figsize=(15,5))
plt.scatter(X_train.TransactionDT,X_train.D15)
plt.title('Original D15')
plt.xlabel('Time')
plt.ylabel('D15')
plt.show()

# normalizácia D stĺpcov
for i in range(1,16):
    if i in [1,2,3,5,9]: continue
    X_train['D'+str(i)] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60) 

# zobrazenie transformovaných D stĺpcov 
plt.figure(figsize=(15,5))
plt.scatter(X_train.TransactionDT,X_train.D15)
plt.title('Transformed D15')
plt.xlabel('Time')
plt.ylabel('D15n')
plt.show()

%%time
# zníženie nárokov na pamäť
for i,f in enumerate(X_train.columns):
    # faktorizácia kategoriálnych premenných
    if (np.str(X_train[f].dtype)=='category')|(X_train[f].dtype=='object'): 
        df_comb = pd.concat([X_train[f],X_test[f]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        if df_comb.max()>32000: print(f,'needs int32')
        X_train[f] = df_comb[:len(X_train)].astype('int16')
        X_test[f] = df_comb[len(X_train):].astype('int16')
    # numerické premenné na pozitívne, NaN hodnoty na -1
    elif f not in ['TransactionAmt','TransactionDT']:
        mn = np.min((X_train[f].min(),X_test[f].min()))
        X_train[f] -= np.float32(mn)
        X_test[f] -= np.float32(mn)
        X_train[f].fillna(-1,inplace=True)
        X_test[f].fillna(-1,inplace=True)

In [ ]:
# frequency encode
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')
        
# label encode
def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')
        
# agregácia skupín
# https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # agregácia pomocou UID pre danú štatistiku 
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')
                
# kombinácia atribútov
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')
    
# agregácia neunikátnych skupín
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

In [ ]:
%%time
# TransactionAmt v centoch 
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
print('cents, ', end='')
# frequency encode: ADDR1, CARD1, CARD2, CARD3, P_EMAILDOMAIN
encode_FE(X_train,X_test,['addr1','card1','card2','card3','P_emaildomain'])
# kombinácia atribútov CARD1+ADDR1, CARD1+ADDR1+P_EMAILDOMAIN
encode_CB('card1','addr1')
encode_CB('card1_addr1','P_emaildomain')
# frequency encode
encode_FE(X_train,X_test,['card1_addr1','card1_addr1_P_emaildomain'])
# agregácia skupín
encode_AG(['TransactionAmt','D9','D11'],['card1','card1_addr1','card1_addr1_P_emaildomain'],['mean','std'],usena=True)

In [ ]:
cols = list( X_train.columns )
cols.remove('TransactionDT')
for c in ['D6','D7','D8','D9','D12','D13','D14']:
    cols.remove(c)
    
for c in ['C3','M5','id_08','id_33']:
    cols.remove(c)
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    cols.remove(c)
for c in ['id_'+str(x) for x in range(22,28)]:
    cols.remove(c)

In [ ]:
#uloženie dát 

X_train.to_csv('train_transaction_po_uprave.csv', index=True)
y_train.to_csv('y_train.csv', index=True)

#***Výber atribútov***

In [ ]:
#kompletne cele data 
X_train = pd.read_csv('train_transaction_po_uprave.csv')
y_train = pd.read_csv('y_train.csv')

In [ ]:
#normálizácia dát pre lepšie fungovanie algoritmov
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
names = X_train.columns
d = scaler.fit_transform(X_train)
scaled_df_train = pd.DataFrame(d, columns=names)
scaled_df_train.tail()

X_train = scaled_df_train
del X_train[X_train.columns[0]]

#pre istotu, keby náhodou boli NaN hodnoty v tabuľke 
X_train = X_train.dropna(axis=1)


In [ ]:
#SelectKBest 
#score_func - funkcia pomocou ktorej vyberáme atribúty, používali sme chi2 a f_classif
#k - počet atribútov, ktoré chceme vybrať. My sme použili počet 30, ale môže sa zmeniť 

#importy k výberu atribútov
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = X_train
#aby sme mali stĺpec isFraud 
y_train = pd.DataFrame(y_train)
y = y_train['isFraud']    

#uvádzame príkazy s oboma funkciami
bestfeatures = SelectKBest(score_func=chi2, k=30)
#bestfeatures = SelectKBest(score_func=f_classif, k=25)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#spojíme predchádzajúce dataframy aby sa ľahšie čítali 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  
#vypíšeme prvých k atribútov
print(featureScores.nlargest(30,'Score'))


In [ ]:
#ExtraTreesClassifier
#max_depth - maximálna hĺbka stromov. My sme použili hodnotu 4, aby sa nevytvárali príliš veľké stromy, ale môže sa použiť aj vyššia hodnota. 

#importy k výberu atribútov
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

X = X_train
#aby sme mali stĺpec isFraud 
y_train = pd.DataFrame(y_train)
y = y_train['isFraud']    

model = ExtraTreesClassifier(max_depth=4)
model.fit(X,y)
print(model.feature_importances_) 
#vizualizácia najlepších atribútov 
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh')
plt.show()
#výpis najlepších atribútov
feat_importances.nlargest(30)

In [ ]:
#opäť načítame celý dataset, aby sme z neho vytvorili nový dataset obsahujúci iba vybrané atribúty 
X_train = pd.read_csv('train_transaction_po_uprave.csv',index_col='TransactionID')
y_train = pd.read_csv('y_train.csv')
X_train = pd.merge(X_train, y_train, on = 'TransactionID')

In [ ]:
#vytvorenie nových datasetov. Pred každým spustením nasledujúcich troch buniek treba 
#spustiť aj predchádzajúcu, aby sme vychádzali vždy z pôvodného datasetu.


#TOP 30 atribútov SelectKBest: chi
X_train = X_train[['id_10', 'id_04', 'id_17', 'id_11', 'id_12', 'id_37', 'D9', 'DeviceType', 'id_06', 'id_09',
                   'id_20', 'id_03', 'id_13', 'id_01', 'id_05', 'D12', 'id_38', 'id_36', 'id_28', 'id_29',
                   'M4', 'D14', 'D7', 'DeviceInfo', 'id_35', 'id_16', 'id_15', 'id_31', 'D6', 'D13', 'isFraud']]
X_train.to_csv('top30_SelectKBest_chi.csv', index=False)

In [ ]:
#TOP 30 atribútov SelectKBest: f_classif
X_train = X_train[['V257', 'V258', 'V228', 'V171', 'V188', 'V176', 'id_17', 'addr2', 'addr1_FE', 'ProductCD',
                   'V198', 'V238', 'V185', 'V221', 'card3', 'id_09', 'V252', 'id_10', 'card3_FE', 'DeviceType',
                   'V261', 'V229', 'id_03', 'V260', 'id_04', 'id_20', 'D12', 'id_37', 'id_12', 'D7' ,'isFraud']]
X_train.to_csv('top30_SelectKBest_f.csv', index=False)

In [ ]:
#TOP 30 atribútov ExtraTreesClassifier: 
X_train = X_train[['V188', 'V257', 'id_17', 'V258', 'addr1_FE', 'id_35', 'id_09', 'addr2', 'id_10', 'V228', 
                   'D9', 'card3', 'V23', 'id_04', 'V86', 'V156', 'V47', 'V187', 'V171', 'id_12', 
                   'M4', 'D7', 'V54', 'D11_card1_std', 'V124', 'V44', 'id_01', 'V65', 'id_38' ,'isFraud']]
X_train.to_csv('top30_ExtraTreesClassifier.csv', index=False)

#***tabGAN***

In [ ]:
!pip install tabgan
#keby nefungovala najnovšia verzia:
#!pip install tabgan==1.1.1 

In [ ]:
X_train = pd.read_csv('top30_ExtraTreesClassifier.csv')

fraud_transaction = X_train
# 75 % dát dáme do tréningovej množiny, zvyšok do testovacej 
X_train = fraud_transaction.sample(frac=0.75, random_state=25)
X_test = fraud_transaction.drop(X_train.index)
y_train = X_train['isFraud']
X_train = X_train.drop(columns=['isFraud'])
X_test = X_test.drop(columns=['isFraud'])

In [ ]:
#!pip install scikit_learn==0.23.2
#je možné, že bude potrebný aj tento install, závisí od prostredia v ktorom spúšťame kódy. Ohľadom tejto veci sme komunikovali aj s autorom tohto algoritmu.

from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np

# random input data

X_train1 = pd.DataFrame(X_train)
X_test1 = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)

train = X_train1
target = y_train
test = X_test1
 
#príklad generátora so základnými nastaveniami 
#new_train, new_target = GANGenerator().generate_data_pipe(train, target, test, )

# príklad generátora s rozšírenými nastaveniami
new_train, new_target = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversaial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(train, target,
                                          test, deep_copy=True, only_adversarial=False, use_adversarial=True)
                                       

new_dataframe = new_train.join(new_target)
new_dataframe.to_csv('model_tabGAN_ETC.csv', index=False)

ContextualVersionConflict: ignored

# **CTGAN**

In [ ]:
from sdv.tabular import CTGAN
import pandas as pd
from ctgan import CTGANSynthesizer 

In [ ]:
#načítanie datasetu 
X_train = pd.read_csv('top30_ExtraTreesClassifier.csv')

#CTGAN so základnými nastaveniami 
ctgan = CTGAN()
ctgan.fit(X_train)
#uloženie modelu 
ctgan.save('model_CTGAN_ETC.pkl')

# **Vytvorenie novej (rozšírenej) tréningovej množiny**

In [ ]:
#v prípade, že chceme použiť výstup z tabGAN

#načítanie pôvodného datasetu 
X_train = pd.read_csv('top30_ExtraTreesClassifier.csv')
y_train = X_train['isFraud']
X_train = X_train.drop(columns=['isFraud'])

#načítame umelú dátovú sadu, treba si dať pozor, aby sme načítali tú, ktorá patrí k datasetu X_train
samples = pd.read_csv('model_tabGAN_ETC.csv')
#vyberieme umelých podvodníkov
podvodnici = samples.loc[samples['isFraud'] == 1]
y_train_podvodnici = podvodnici['isFraud'].copy()
podvodnici = podvodnici.drop(columns=['isFraud'])

from sklearn.model_selection import train_test_split
#rozdelíme pôvodné dáta na train/test 
X = X_train
y = y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#vytvoríme novy train dataset s umelými podvodníkmi, test neobsahuje podvodné transakcie, ten dataset necháme v pôvodnom stave
X_train = pd.concat([X_train, podvodnici])
y_train = pd.concat([y_train, y_train_podvodnici])

#pomiešame dáta
X_train = X_train.join(y_train).sample(frac = 1)
y_train = X_train['isFraud'].copy()
X_train = X_train.drop(columns=['isFraud'])


In [ ]:
#v prípade, že chceme použiť výstup z CTGAN

#načítanie pôvodného datasetu 
X_train = pd.read_csv('top30_ExtraTreesClassifier.csv')
y_train = X_train['isFraud']
X_train = X_train.drop(columns=['isFraud'])

#načítame model CTGAN, treba si dať pozor, aby sme načítali ten, ktorý patrí k datasetu X_train
ctgan = CTGAN.load('model_CTGAN_ETC.pkl')

#generujeme umelé dáta
samples = ctgan.sample(10000)
#vyberieme umelých podvodníkov
podvodnici = samples.loc[samples['isFraud'] == 1]
y_train_podvodnici = podvodnici['isFraud'].copy()
podvodnici = podvodnici.drop(columns=['isFraud'])

from sklearn.model_selection import train_test_split
#rozdelíme pôvodné dáta na train/test 
X = X_train
y = y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#vytvoríme novy train dataset s umelými podvodníkmi, test neobsahuje podvodné transakcie, ten dataset necháme v pôvodnom stave
X_train = pd.concat([X_train, podvodnici])
y_train = pd.concat([y_train, y_train_podvodnici])

#pomiešame dáta
X_train = X_train.join(y_train).sample(frac = 1)
y_train = X_train['isFraud'].copy()
X_train = X_train.drop(columns=['isFraud'])


# **Rozhodovacie stromy**

In [ ]:
#načítanie datasetu - bez umelých podvodníkov 
X = pd.read_csv('top30_ExtraTreesClassifier.csv')
#cieľová premenná 
y = X['isFraud']
X = X.drop(columns=['isFraud'])

#vytvorenie tréningovej a testovacej množiny, do tréningovej množiny sme vložili 67 % dát, zvyšok do testovacej, tento parameter tiež môžeme zmeniť. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#použitie datasetu - s umelými podvodníkmi z časti Vytvorenie novej (rozšírenej) tréningovej množiny. 
#Tieto datasety by sa dali aj uložiť, kvôli šetreniu miesta sme ich neukladali stále zvlášť 

#pre istotu, keby náhodou boli NaN hodnoty v tabuľke, náhodný les s nimi nevie pracovať 
X_train = X_train.dropna(axis=1)

y = y_train
X = X_train

#vytvorenie tréningovej a testovacej množiny, do tréningovej množiny sme vložili 67 % dát, zvyšok do testovacej, tento parameter tiež môžeme zmeniť. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#Náhodný les 
#použili sme 1000 stromov, ale tento parameter sa môže podľa potreby meniť, nám sa nepodarilo zväčšením počtu stromov dosiahnuť lepšie výsledky (optimálny počet bol ešte nižší) 

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

#vytvoríme klasifikátor
clf=RandomForestClassifier(n_estimators=1000)

#trénovanie modelu
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

# pozrieme sa na presnosť modelu 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

#***XGBoost***

In [ ]:
#túto bunku používať v prípade, že chceme pracovať s uloženým datasetom

#načítanie datasetu 
X = pd.read_csv('top30_ExtraTreesClassifier.csv')
#cieľová premenná 
y = X['isFraud']
X = X.drop(columns=['isFraud'])

#vytvorenie tréningovej a testovacej množiny, do tréningovej množiny sme vložili 67 % dát, zvyšok do testovacej, tento parameter tiež môžeme zmeniť. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#túto bunku používať v prípade použitia datasetu s umelými podvodníkmi z časti Vytvorenie novej (rozšírenej) tréningovej množiny. 
#Tieto datasety by sa dali aj uložiť, kvôli šetreniu miesta sme ich neukladali stále zvlášť, názvy premenných sú nastavené tak, aby sa rovno dala použiť táto bunka 

y = y_train
X = X_train

#vytvorenie tréningovej a testovacej množiny, do tréningovej množiny sme vložili 67 % dát, zvyšok do testovacej, tento parameter tiež môžeme zmeniť. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#algoritmus XGBoost
#môžeme zmeniť počet stromov, ich maximálnu hĺbku, učiaci parameter a iné. 
#ak chceme použiť GPU, tak treba odkomentovať riadok pod USE GPU a zakomentovať riadku pod USE CPU

import xgboost as xgb
print("XGBoost version:", xgb.__version__)

if BUILD95:
    clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        eval_metric='mae',
        # USE CPU
        nthread=4,
        tree_method='hist',
        seed = 42 
        # USE GPU
        #tree_method='gpu_hist' 
    )
    h = clf.fit(X_train, y_train, 
        eval_set=[(X_test,y_test)],
        verbose=50, early_stopping_rounds=100)

XGBoost version: 1.5.1


/home/lubomir_antoni/.conda/envs/diplomove_vypocty/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mae:0.49111
[50]	validation_0-mae:0.21764
[100]	validation_0-mae:0.11657
[150]	validation_0-mae:0.07838
[200]	validation_0-mae:0.06342
[250]	validation_0-mae:0.05722
[300]	validation_0-mae:0.05436
[350]	validation_0-mae:0.05276
[400]	validation_0-mae:0.05182
[450]	validation_0-mae:0.05114
[500]	validation_0-mae:0.05063
[550]	validation_0-mae:0.05014
[600]	validation_0-mae:0.04969
[650]	validation_0-mae:0.04937
[700]	validation_0-mae:0.04902
[750]	validation_0-mae:0.04871
[800]	validation_0-mae:0.04845
[850]	validation_0-mae:0.04821
[900]	validation_0-mae:0.04800
[950]	validation_0-mae:0.04773
[1000]	validation_0-mae:0.04750
[1050]	validation_0-mae:0.04728
[1100]	validation_0-mae:0.04709
[1150]	validation_0-mae:0.04689
[1200]	validation_0-mae:0.04670
[1250]	validation_0-mae:0.04653
[1300]	validation_0-mae:0.04637
[1350]	validation_0-mae:0.04620
[1400]	validation_0-mae:0.04604
[1450]	validation_0-mae:0.04586
[1500]	validation_0-mae:0.04573
[1550]	validation_0-mae:0.04558